In [2]:
#ADVERTENCIA
#Este bloque de instrucciones solo hay que ejecutarlo si el entorno
#Anaconda3 Jupyter donde se ejecuta el programa de creación de los ficheros
#no estuvieran instalados estos móduloa

#!pip uninstall -y numpy
#!pip install "numpy>=2.0.0,<3.0.0"
#!pip install "thinc>=8.3.0,<9.0.0"
#!pip install "spacy==3.8.5"
#!pip install --upgrade "numpy>=2.0.0,<3.0.0" thinc spacy h5py
#!python -m spacy download es_core_news_sm

In [17]:
import re
import os
import csv
import spacy
import json
from collections import Counter

# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

# Increase the maximum length that spaCy can handle
nlp.max_length = 3000000 # Increased to 3,000,000

#Inicio cálculo indice legibilidad según Fernández Huerta
#ïndice legilibilidad Fernández Huerta
def calcular_legibilidad_fernandez_huerta(texto):
    doc = nlp(texto)
    # Contadores
    total_silabas = 0
    total_palabras = 0
    total_oraciones = 0

    # Procesar cada oración
    for oracion in doc.sents:
        total_oraciones += 1
        for token in oracion:
            if not token.is_punct and not token.is_space:
                total_palabras += 1
                total_silabas += contar_silabas(token.text)

    # Calcular promedios (por 100 palabras, como en la fórmula original)
    P = (total_silabas / total_palabras) * 100  # Sílabas por 100 palabras
    F = (total_palabras / total_oraciones)      # Palabras por oración

    # Aplicar fórmula Fernández Huerta
    L = 206.84 - (0.60 * P) - (1.02 * F)
    return {
        "puntuacion": round(L, 2),
        "metricas": {
            "sílabas_por_100_palabras": round(P, 2),
            "palabras_por_oracion": round(F, 2),
        }
    }

#Fin cálculo indice legibilidad según Fernández Huerta

#Inicio cálculo contar sílabas
def contar_silabas(palabra):
    """Función simplificada para contar sílabas en español"""
    palabra = palabra.lower()
    vocales = "aeiouáéíóúü"
    silabas = 0
    for letra in palabra:
        if letra in vocales:
            silabas += 1
    return max(1, silabas)  # Mínimo 1 sílaba
#fin cálculo contar sílabas

def analizar_capitulos(ruta_capitulos, archivo_salida):
    # Lista para almacenar todos los datos
    datos = []

    # Campos del CSV
    campos = [
        'Capitulo',
        'Total caracteres',
        'Total palabras',
        'Longitud media frases',
        'Palabras lexicas',
        'Palabras funcionales',
        'Densidad lexica',
        'Indice legilibilidad F.H.',
        'Frecuencia "Quijote"',
        'Frecuencia "Sancho"'
    ]

    # Procesar cada archivo de capítulo
    for archivo in os.listdir(ruta_capitulos):
        if archivo.endswith('.txt'):
            ruta_completa = os.path.join(ruta_capitulos, archivo)
            with open(ruta_completa, 'r', encoding='utf-8') as f:
                texto = f.read()
#sustituir guiones pegados a las palabras por espacio, pues en otro caso no contaba esas palabras
#por ejemplo "Quijote-" con contaba con ocurrencia de "Quijote"
                texto = texto.replace('-',' ')
                doc = nlp(texto)

                # Calcular métricas
                total_caracteres = len(texto)
                total_palabras = len([token for token in doc if not token.is_punct and token.is_alpha])
                longitud_media_frases = sum(len(sent) for sent in doc.sents) / len(list(doc.sents)) if doc.sents else 0

                palabras_lexicas = len([token for token in doc if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV']])
                palabras_funcionales = len([token for token in doc if token.pos_ in ['DET', 'ADP', 'CONJ', 'CCONJ', 'SCONJ', 'PRON']])
                densidad_lexica = palabras_lexicas / total_palabras if total_palabras > 0 else 0
                densidad_lexica2 = palabras_lexicas / palabras_funcionales if palabras_funcionales > 0 else 0

                quijote_count = len([token for token in doc if token.text.lower() == 'quijote'])
                sancho_count = len([token for token in doc if token.text.lower() == 'sancho'])

#Cálculo
                resultado_il_fh = calcular_legibilidad_fernandez_huerta(texto)
                indice_legilibilidad_fh = resultado_il_fh["puntuacion"]

# Añadir datos a la lista
#cambiar "." por "," en decimales
                longitud_media_frases = str("%.2f" % round(longitud_media_frases, 2)).replace('.', ',')
                densidad_lexica = str("%.2f" % round(densidad_lexica, 2)).replace('.', ',')
                indice_legilibilidad_fh = str("%.2f" % round(indice_legilibilidad_fh, 2)).replace('.', ',')
                datos.append([
                    archivo,
                    total_caracteres,
                    total_palabras,
                    longitud_media_frases,
                    palabras_lexicas,
                    palabras_funcionales,
                    densidad_lexica,
                    indice_legilibilidad_fh,
                    quijote_count,
                    sancho_count])

    carpeta = os.path.join('Quijote', 'metricas_capitulos')
    ruta_completa_fich = os.path.join(carpeta, 'metricas_capitulos')
    archivo_salida = ruta_completa_fich
    # Escribir el archivo CSV
    with open(archivo_salida + '.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(campos)
        writer.writerows(datos)

# Escribir la misma información en un fichero JSON para utilizar luego con ALTAIR
    cabecera = campos
    filas = datos[0:]
    datos_json = [dict(zip(cabecera, fila)) for fila in filas]
    with open(archivo_salida + '.json', 'w') as archivo:
      json.dump(datos_json, archivo, indent=4)



# Configuración de rutas
#ruta_capitulos = "/content/drive/MyDrive/_MHD-UNED/09-Análisis Estilométrico y de Texto/Quijote/Quijote por capitulos-solo-uno/"
#ruta_capitulos = 'Quijote'
ruta_capitulos = 'Quijote/quijote por capitulos'
archivo_salida = "metricas_capitulos"
#archivo_salida_csv = "/content/drive/MyDrive/_MHD-UNED/09-Análisis Estilométrico y de Texto/Quijote/metricas_capitulos-20250603.csv"
#archivo_salida_json = "/content/drive/MyDrive/_MHD-UNED/09-Análisis Estilométrico y de Texto/Quijote/metricas_capitulos-20250603.json"
#archivo_salida = "/content/drive/MyDrive/_MHD-UNED/09-Análisis Estilométrico y de Texto/Quijote/metricas_capitulos-1-c08.csv"

# Ejecutar el análisis
os.makedirs('Quijote/metricas_capitulos/', exist_ok=True)
analizar_capitulos(ruta_capitulos, archivo_salida)
print(f"Análisis completado. Los resultados se han guardado en: {archivo_salida}")



Análisis completado. Los resultados se han guardado en: metricas_capitulos
